### Wrapper Class
* multiple inheritance? Cannot!
  * `ListIter` from `Iter` and `List`
* OOP < type class

In [11]:
import scala.annotation.tailrec

abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
abstract class Iterable[A] {
    def iter: Iter[A]
}
// behave as it is inherited
// composition inheritance from List class
class ListIter[A](val list: List[A]) extends Iter[A] {
    def getValue = list.headOption
    def getNext = new ListIter(list.tail)
    def printList: Any = ListIter.printIter(this)
}
object ListIter {
    @tailrec
    def printIter[A](x: ListIter[A]): Any = {
        x.getValue match {
            case None => println()
            case Some(v) => {
                print(v + " ")
                printIter(x.getNext)
            }
        }
    }
}
(new ListIter(List(4, 3, 2, 1))).printList
ListIter.printIter(new ListIter(List(1, 2, 3, 4)))

4 3 2 1 
1 2 3 4 


import scala.annotation.tailrec


defined class Iter
defined class Iterable
defined class ListIter
defined object ListIter
res10_5: Any = ()
res10_6: Any = ()

#### * ***Program against interfaces***
###### use inheritance when implementing on interface,
###### 인터페이스를 사용할 때만 상속을 사용

#### * ***Composition over inheritance***
###### use composition when reusing functionalities, 
###### 기능을 상속 받을 때는 인자로 받아서 사용

In [12]:
import scala.annotation.tailrec
def sumElements[A](f: A=>Int)(lst: Iter[A]): Int = {
    @tailrec
    def sumIter(x: Iter[A], acc: Int): Int = {
        x.getValue match {
            case None => acc
            case Some(v) => sumIter(x.getNext, acc + f(v))
        }
    }
    sumIter(lst, 0)
}
def sumElementsGen[A](f: A=>Int)(lst: Iterable[A]): Int =
    sumElements(f)(lst.iter)

// sumElements((x:Int)=>x)(List(1,2,3,4))
sumElements((x:Int)=>x)(new ListIter(List(1,2,3,4)))

import scala.annotation.tailrec

defined function sumElements
defined function sumElementsGen
res11_3: Int = 10

In [13]:
List(1,2,3,4)
1::2::3::4::Nil

val x = 1::2::3::Nil
val y = 4::5::Nil
x ++ y

res12_0: List[Int] = List(1, 2, 3, 4)
res12_1: List[Int] = List(1, 2, 3, 4)
x: List[Int] = List(1, 2, 3)
y: List[Int] = List(4, 5)
res12_4: List[Int] = List(1, 2, 3, 4, 5)

### List Syntax
* `List(elements)`: list constructor
* `++`: list append, same as `.++()`
* `::`: value append, same as `.::()`

In [14]:
sealed abstract class MyList[A] extends Iter[A] {
    def ++(lst: MyList[A]): MyList[A]
}
case class MyNil[A]() extends MyList[A] {
    def getValue = None
    def getNext = throw new Exception("...")
    def ++(lst: MyList[A]) = lst
}
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A] {
    def getValue = Some(hd)
    def getNext = tl
    def ++(lst: MyList[A]) = MyCons(hd, tl++lst)
}
object MyList {
    def ::[A](v1: A, v2: A): MyList[A] = MyCons(v1, MyCons(v2, MyNil()))
    def ::[A](v: A, lst: MyList[A]): MyList[A] = MyCons(v, lst)
    def ++[A](lst1: MyList[A], lst2: MyList[A]): MyList[A] = lst1++lst2
}
val x = MyCons(2, MyCons(3, MyNil()))
val y = MyCons(2, MyCons(3, MyNil()))
MyList.::(1, 2)
MyList.::(1, MyNil[Int]())
MyList.++(x, y)

defined class MyList
defined class MyNil
defined class MyCons
defined object MyList
x: MyCons[Int] = MyCons(2, MyCons(3, MyNil()))
y: MyCons[Int] = MyCons(2, MyCons(3, MyNil()))
res13_6: MyList[Int] = MyCons(1, MyCons(2, MyNil()))
res13_7: MyList[Int] = MyCons(1, MyNil())
res13_8: MyList[Int] = MyCons(2, MyCons(3, MyCons(2, MyCons(3, MyNil()))))

#### Compatibility between *ListIter* and *List* 
  * `ListIter.list` & `new ListIter(List)`

In [15]:
sealed abstract class MyTree[A] extends Iterable[A] {
    override def iter: ListIter[A]
}
case class Empty[A]() extends MyTree[A] {
    val iter: ListIter[A] = new ListIter(Nil)
}
case class Node[A](value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A] {
    val iter: ListIter[A] =
        // new ListIter(value::(left.iter.list ++ right.iter.list))
        // new ListIter(left.iter.list ++ (value::right.iter.list))
        new ListIter(left.iter.list ++ right.iter.list ++ (value::Nil))
}
object MyTree {
    def printTree[A](t: MyTree[A]): Any = {
        ListIter.printIter(t.iter)
    }
}

defined class MyTree
defined class Empty
defined class Node
defined object MyTree

In [16]:
val t: MyTree[Int] = 
    Node(3, Node(4,Node(2,Empty(),Empty()),
    Node(3,Empty(),Empty())),
    Node(5,Empty(),Empty()))
MyTree.printTree(t)

2 3 4 5 3 


t: MyTree[Int] = Node(
  3,
  Node(4, Node(2, Empty(), Empty()), Node(3, Empty(), Empty())),
  Node(5, Empty(), Empty())
)
res15_1: Any = ()

In [17]:
sumElementsGen((x:Int)=>x)(t)

res16: Int = 17

### Abstract Type
* Internally defined user type
* How to eliminate?

In [1]:
abstract class Iterable[A] {
    type iter_t
    def iter: iter_t
    def getValue(i: iter_t): Option[A]
    def getNext(i: iter_t): iter_t
}

defined class Iterable

In [2]:
import scala.annotation.tailrec
def sumElements[A](f: A=>Int)(xs: Iterable[A]): Int = {
    @tailrec
    def sumIter(i: xs.iter_t, acc: Int): Int = {
        xs.getValue(i) match {
            case None => acc
            case Some(v) => sumIter(xs.getNext(i), acc+f(v))
        }
    }
    sumIter(xs.iter, 0)
}

import scala.annotation.tailrec

defined function sumElements

* MyList example

In [3]:
class MyList[A](val iter: List[A]) extends Iterable[A] {
    type iter_t = List[A]
    def getValue(i: List[A]): Option[A] = i.headOption
    def getNext(i: List[A]): List[A] = i.tail
}
sumElements((x:Int)=>x)(new MyList(1::2::Nil))

defined class MyList
res2_1: Int = 3

* MyTree example

In [16]:
sealed abstract class MyTree[A] extends Iterable[A] {
    type iter_t = List[A]
    def getValue(i: List[A]): Option[A] = i.headOption
    def getNext(i: List[A]): List[A] = i.tail
}
case class Empty[A]() extends MyTree[A] {
    val iter : List[A] = Nil
}
case class Node[A](value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A] {
    val iter = value :: (left.iter ++ right.iter) // Pre-order
    //val iter = left.iter ++ (value :: right.iter) // In-order
    //val iter = left.iter ++ (right.iter ++ List(value)) // Post-order
}

defined class MyTree
defined class Empty
defined class Node

In [17]:
val t : MyTree[Int] =
    Node(3, Node(4,Node(2,Empty(),Empty()),
    Node(3,Empty(),Empty())),
    Node(5,Empty(),Empty()))
sumElements((x:Int)=>x)(t)

t: MyTree[Int] = Node(
  3,
  Node(4, Node(2, Empty(), Empty()), Node(3, Empty(), Empty())),
  Node(5, Empty(), Empty())
)
res16_1: Int = 17

### Has Element in Iterable
* no implementations in abstract class
* but allows for this functionality `HasElements`
  * never override (danger)

In [31]:
// how can we not passing 'eq' but later?
abstract class IterableHE[A](eq: (A,A) => Boolean) extends Iterable[A] {
    def hasElement(a: A) : Boolean = {
        def hasElementIter(i: iter_t) : Boolean =
            getValue(i) match {
                case None => false
                case Some(n) =>
                if (eq(a,n)) true
                else hasElementIter(getNext(i))
            }
        hasElementIter(iter)
    }
}

defined class IterableHE

In [32]:
sealed abstract class MyTree[A](eq:(A,A)=>Boolean) extends IterableHE[A](eq) {
    type iter_t = List[A]
    def getValue(i : List[A]) : Option[A] = i.headOption
    def getNext(i: List[A]) : List[A] = i.tail
}
case class Empty[A](eq: (A,A)=>Boolean) extends MyTree[A](eq) {
    val iter : List[A] = Nil
}
case class Node[A](eq: (A,A)=>Boolean, value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A](eq) {
    val iter : List[A] = value :: (left.iter ++ right.iter)
}

defined class MyTree
defined class Empty
defined class Node

In [33]:
val Ieq = (x:Int, y:Int) => x == y
val IEmpty = Empty(Ieq)
def INode(n: Int, t1: MyTree[Int], t2: MyTree[Int]) = Node(Ieq, n, t1, t2)
val t : MyTree[Int] =
    INode(3, INode(4,INode(2,IEmpty,IEmpty),
    INode(3,IEmpty,IEmpty)),
    INode(5,IEmpty,IEmpty))
sumElements((x:Int)=>x)(t)
t.hasElement(5)
t.hasElement(10)

Ieq: (Int, Int) => Boolean = ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae
IEmpty: Empty[Int] = Empty(
  ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae
)
defined function INode
t: MyTree[Int] = Node(
  ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae,
  3,
  Node(
    ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae,
    4,
    Node(
      ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae,
      2,
      Empty(ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae),
      Empty(ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae)
    ),
    Node(
      ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae,
      3,
      Empty(ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae),
      Empty(ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae)
    )
  ),
  Node(
    ammonite.$sess.cmd32$Helper$$Lambda$2325/1769564542@4a6ec8ae,
    5,
    Empty(ammonite.$sess.cmd32$Helper$$Lambda$2325

#### Put `eq` function in interface of abstract class

In [34]:
abstract class IterableHE[A] extends Iterable[A] {
    def eq(a:A, b:A) : Boolean
    def hasElement(a: A) : Boolean = {
        def hasElementIter(i: iter_t) : Boolean =
            getValue(i) match {
                case None => false
                case Some(n) =>
                    if (eq(a,n)) true
                    else hasElementIter(getNext(i))
            }
        hasElementIter(iter)
    }
}

defined class IterableHE

In [37]:
sealed abstract class MyTree[A] extends IterableHE[A] {
    type iter_t = List[A]
    def getValue(i : List[A]) : Option[A] = i.headOption
    def getNext(i: List[A]) : List[A] = i.tail
}
// why do we have to pass '_eq'
case class Empty[A](_eq: (A,A)=>Boolean) extends MyTree[A] {
    def eq(a: A, b: A) = _eq(a, b)
    val iter : List[A] = Nil
}
case class Node[A](_eq: (A,A)=>Boolean, value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A] {
    def eq(a: A, b: A) = _eq(a, b)
    val iter : List[A] = value :: (left.iter ++ right.iter)
}

defined class MyTree
defined class Empty
defined class Node

In [36]:
val Ieq = (x:Int, y:Int) => x == y
val IEmpty = Empty(Ieq)
def INode(n: Int, t1: MyTree[Int], t2: MyTree[Int]) = Node(Ieq, n, t1, t2)
val t : MyTree[Int] =
    INode(3, INode(4,INode(2,IEmpty,IEmpty),
    INode(3,IEmpty,IEmpty)),
    INode(5,IEmpty,IEmpty))
sumElements((x:Int)=>x)(t)
t.hasElement(5)
t.hasElement(10)

Ieq: (Int, Int) => Boolean = ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e
IEmpty: Empty[Int] = Empty(
  ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e
)
defined function INode
t: MyTree[Int] = Node(
  ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e,
  3,
  Node(
    ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e,
    4,
    Node(
      ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e,
      2,
      Empty(ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e),
      Empty(ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e)
    ),
    Node(
      ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e,
      3,
      Empty(ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e),
      Empty(ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e)
    )
  ),
  Node(
    ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e,
    5,
    Empty(ammonite.$sess.cmd35$Helper$$Lambda$2343/2021902230@cb810e),
 

### Implementation (Needs of type class)
* interface unit: object (OOP)
* interface unit: type (type class)